# Question 1

In [74]:
import gurobipy as gp
from gurobipy import GRB

In [75]:
TruckCar, cost = gp.multidict(
{
    (1, 1): 276,
    (1, 2): 497,
    (1, 3): 251,
    (1, 4): 364,
    (2, 1): 179,
    (2, 2): 375,
    (2, 3): 298,
    (2, 4): 190,
    (3, 1): 150,
    (3, 2): 475,
    (3, 3): 344,
    (3, 4): 492,
    (4, 1): 97,
    (4, 2): 163,
    (4, 3): 285,
    (4, 4): 185,
    (5, 1): 305,
    (5, 2): 150,
    (5, 3): 225,
    (5, 4): 165
})

capacity = {
    1: 2,  
    2: 1,  
    3: 1,  
    4: 1,  
    5: 2   
}

In [76]:
mTruckCar = gp.Model('TruckCar')

In [77]:
assign = mTruckCar.addVars(TruckCar,vtype='B', obj=cost, name='TruckCar') 


In [78]:
for i in range(1, 6):
    mTruckCar.addConstr(
        sum(assign[i, j] for i,j in TruckCar.select(i, '*')) <= capacity[i],
        name="truck_capacity"
)
    
for j in range(1, 5):
    mTruckCar.addConstr(
        sum(assign[i, j] for i,j in TruckCar.select('*', j)) == 1,
        name="car_demand"
)

In [79]:
# Verify model formulation

mTruckCar.write('TruckCar.lp')

In [80]:
mTruckCar.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xf4a71cbf
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1490.0000000
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 918.0000000

Root relaxation: objective 6.620000e+02, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*

We want to print the variables with a value of greater than 0 because this is the tuple that gurobi's optimization says we should take

In [81]:
for x in mTruckCar.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total cost: ' + str(mTruckCar.objVal))
print('Total profit: ' + str(2400-mTruckCar.objVal))

TruckCar[2,4] 1.0
TruckCar[4,1] 1.0
TruckCar[5,2] 1.0
TruckCar[5,3] 1.0
Total cost: 662.0
Total profit: 1738.0


# Question 2

In [82]:
AssetYear, spend = gp.multidict(
{
    (1, 1): 35,  # Car Year 1
    (1, 2): 37,  # Car Year 2
    (1, 3): 39,  # Car Year 3
    (1, 4): 42,  # Car Year 4
    (1, 5): 45,  # Car Year 5
    (2, 1): 16,  # Piano Year 1
    (2, 2): 17,  # Piano Year 2
    (2, 3): 18,  # Piano Year 3
    (2, 4): 19,  # Piano Year 4
    (2, 5): 20,  # Piano Year 5
    (3, 1): 125, # Necklace Year 1
    (3, 2): 130, # Necklace Year 2
    (3, 3): 136, # Necklace Year 3
    (3, 4): 139, # Necklace Year 4
    (3, 5): 144, # Necklace Year 5
    (4, 1): 25,  # Desk Year 1
    (4, 2): 27,  # Desk Year 2
    (4, 3): 29,  # Desk Year 3
    (4, 4): 30,  # Desk Year 4
    (4, 5): 33,  # Desk Year 5
    (5, 1): 40,  # Golf Clubs Year 1
    (5, 2): 43,  # Golf Clubs Year 2
    (5, 3): 46,  # Golf Clubs Year 3
    (5, 4): 50,  # Golf Clubs Year 4
    (5, 5): 52,  # Golf Clubs Year 5
    (6, 1): 5,   # Humidor Year 1
    (6, 2): 7,   # Humidor Year 2
    (6, 3): 8,   # Humidor Year 3
    (6, 4): 10,  # Humidor Year 4
    (6, 5): 11   # Humidor Year 5
})



In [83]:
mAssetYear = gp.Model('AssetYear')
assign = mAssetYear.addVars(AssetYear, vtype='B', name='assign')

# Objective function
money = gp.quicksum(spend[i,j]*assign[i,j] for i,j in AssetYear )
mAssetYear.setObjective(money, GRB.MAXIMIZE)

for j in range(1, 6):
    mAssetYear.addConstr(
        sum(  spend[i,j] * assign[i, j] for i,j in AssetYear.select("*", j)) >= 30,
        name="spend_year"
)
    
for i in range(1, 7):
    mAssetYear.addConstr(
        sum( assign[i, j] for i,j in AssetYear.select(i,'*')) == 1,
        name="asset_demand"
)
    
# Verify model formulation
mAssetYear.write('AssetYear.lp')    

mAssetYear.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11 rows, 30 columns and 60 nonzeros
Model fingerprint: 0x42417346
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 274.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 11 rows, 28 columns, 56 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)

Root relaxation: objective 2.750000e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0   

In [84]:
for x in mAssetYear.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total spend: ' + str(mAssetYear.objVal*1000))

assign[1,1] 1.0
assign[2,5] 1.0
assign[3,3] 1.0
assign[4,4] 1.0
assign[5,2] 1.0
assign[6,5] 1.0
Total spend: 275000.0


# Question 3

In [85]:

regionToRegeion, time = gp.multidict(
{
    (1, 1): 0,  # Region 1 to Region 1
    (1, 2): 4,  # Region 1 to Region 2
    (1, 3): 3,  # Region 1 to Region 3
    (1, 4): 6,  # Region 1 to Region 4
    (1, 5): 6,  # Region 1 to Region 5
    (1, 6): 5,  # Region 1 to Region 6
    (2, 1): 4,  # Region 2 to Region 1
    (2, 2): 0,  # Region 2 to Region 2
    (2, 3): 7,  # Region 2 to Region 3
    (2, 4): 5,  # Region 2 to Region 4
    (2, 5): 5,  # Region 2 to Region 5
    (2, 6): 6,  # Region 2 to Region 6
    (3, 1): 3,  # Region 3 to Region 1
    (3, 2): 7,  # Region 3 to Region 2
    (3, 3): 0,  # Region 3 to Region 3
    (3, 4): 4,  # Region 3 to Region 4
    (3, 5): 3,  # Region 3 to Region 5
    (3, 6): 5,  # Region 3 to Region 6
    (4, 1): 6,  # Region 4 to Region 1
    (4, 2): 5,  # Region 4 to Region 2
    (4, 3): 4,  # Region 4 to Region 3
    (4, 4): 0,  # Region 4 to Region 4
    (4, 5): 7,  # Region 4 to Region 5
    (4, 6): 5,  # Region 4 to Region 6
    (5, 1): 6,  # Region 5 to Region 1
    (5, 2): 5,  # Region 5 to Region 2
    (5, 3): 3,  # Region 5 to Region 3
    (5, 4): 7,  # Region 5 to Region 4
    (5, 5): 0,  # Region 5 to Region 5
    (5, 6): 2,  # Region 5 to Region 6
    (6, 1): 5,  # Region 6 to Region 1
    (6, 2): 6,  # Region 6 to Region 2
    (6, 3): 5,  # Region 6 to Region 3
    (6, 4): 5,  # Region 6 to Region 4
    (6, 5): 2,  # Region 6 to Region 5
    (6, 6): 0   # Region 6 to Region 6
})

pop = {
    1: 21,  
    2: 35,  
    3: 15,  
    4: 60,  
    5: 20,
    6: 37   
}

In [86]:
# Transform the time values based on the condition
transformed_time = {(i, j): (1 if time[i, j] >= 4 else 0) for i, j in time}
transformed_time

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 0,
 (1, 4): 1,
 (1, 5): 1,
 (1, 6): 1,
 (2, 1): 1,
 (2, 2): 0,
 (2, 3): 1,
 (2, 4): 1,
 (2, 5): 1,
 (2, 6): 1,
 (3, 1): 0,
 (3, 2): 1,
 (3, 3): 0,
 (3, 4): 1,
 (3, 5): 0,
 (3, 6): 1,
 (4, 1): 1,
 (4, 2): 1,
 (4, 3): 1,
 (4, 4): 0,
 (4, 5): 1,
 (4, 6): 1,
 (5, 1): 1,
 (5, 2): 1,
 (5, 3): 0,
 (5, 4): 1,
 (5, 5): 0,
 (5, 6): 0,
 (6, 1): 1,
 (6, 2): 1,
 (6, 3): 1,
 (6, 4): 1,
 (6, 5): 0,
 (6, 6): 0}

In [87]:
mAmb = gp.Model('Ambulance')
assign = mAmb.addVars(pop, vtype='B', name='assign')
region_in4 = mAmb.addVars(pop, vtype='B', name='region_in4')

# Objective function
tot = gp.quicksum(pop[r]*region_in4[r] for r in pop )
mAmb.setObjective(tot, GRB.MAXIMIZE)

mAmb.addConstr(
        sum(assign[i] for i in pop) == 2,
        name="tot_ambulance"
)

# Transform the time values based on the condition
transformed_time = {(i, j): (1 if time[i, j] <= 4 else 0) for i, j in time}

for i in range(1, 7):
    mAmb.addConstr(
        sum( transformed_time[i, j] * assign[j] 
            for i,j in regionToRegeion.select(i,'*')) >= region_in4[i],
        name="num_amb"
)
    
# Verify model formulation
mAmb.write('Ambulance.lp')    

mAmb.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 12 columns and 28 nonzeros
Model fingerprint: 0x48158cce
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 132.0000000
Presolve removed 7 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 153 132 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.530000000000e+02, best bound 1.530000000000e+02, gap 0.0000%


In [88]:
for x in mAmb.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total population: ' + str(mAmb.objVal*1000))

assign[3] 1.0
assign[6] 1.0
region_in4[1] 1.0
region_in4[3] 1.0
region_in4[4] 1.0
region_in4[5] 1.0
region_in4[6] 1.0
Total population: 153000.0


D.  How many ambulances would be required to provide coverage within four minutes to all residents?

New objective and constraint are in the pdf

In [89]:
mAmb = gp.Model('Ambulance')
assign = mAmb.addVars(pop, vtype='B', name='assign')
region_in4 = mAmb.addVars(pop, vtype='B', name='region_in4')

# Objective function
tot_amb = gp.quicksum(assign[r] for r in pop )
mAmb.setObjective(tot_amb, GRB.MINIMIZE)


# Transform the time values based on the condition
transformed_time = {(i, j): (1 if time[i, j] <= 4 else 0) for i, j in time}

for i in range(1, 7):
    mAmb.addConstr(
        sum( transformed_time[i, j] * assign[j] 
            for i,j in regionToRegeion.select(i,'*')) >= region_in4[i],
        name="num_amb"
)

# add new const
mAmb.addConstr( sum(pop[r]*region_in4[r] for r in pop ) == sum(pop.values()),  name="tot_pop")

# Verify model formulation
mAmb.write('Ambulance.lp')    

mAmb.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 12 columns and 28 nonzeros
Model fingerprint: 0x519dd082
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 7 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [90]:
for x in mAmb.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total ambulance: ' + str(mAmb.objVal))

assign[1] 1.0
assign[3] 1.0
assign[5] 1.0
region_in4[1] 1.0
region_in4[2] 1.0
region_in4[3] 1.0
region_in4[4] 1.0
region_in4[5] 1.0
region_in4[6] 1.0
Total ambulance: 3.0


location 1,3,5 should be 3 locations

E. Suppose the county wants to locate three ambulances in such a way to provide coverage to all 
residents within four minutes and maximize the redundancy in the system. (Assume redundancy means 
being able to provide service by one or more ambulances within four minutes.) Where should the 
ambulances be located? 

change the assign to integer so we may obtain more than 1 ambulance for 1 region

In [97]:
mAmb = gp.Model('Ambulance')
assign = mAmb.addVars(pop, vtype='I', name='assign')
region_in4 = mAmb.addVars(pop, vtype='B', name='region_in4')

# Objective function
tot_amb = gp.quicksum(assign[r] for r in pop )
mAmb.setObjective(tot_amb, GRB.MINIMIZE)

mAmb.addConstr(
        sum(assign[i] for i in pop) == 3,
        name="tot_ambulance"
)

# Transform the time values based on the condition
transformed_time = {(i, j): (1 if time[i, j] <= 4 else 0) for i, j in time}

for i in range(1, 7):
    mAmb.addConstr(
        sum( transformed_time[i, j] * assign[j] 
            for i,j in regionToRegeion.select(i,'*')) >= region_in4[i],
        name="num_amb"
)

# add new const
mAmb.addConstr( sum(pop[r]*region_in4[r] for r in pop ) == sum(pop.values()),  name="tot_pop")

# Verify model formulation
mAmb.write('Ambulance.lp')    

mAmb.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 12 columns and 34 nonzeros
Model fingerprint: 0x0e18859d
Variable types: 0 continuous, 12 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [98]:
for x in mAmb.getVars():
    if (x.x != 0):
        print(x.varName, x.x)
print('Total ambulance: ' + str(mAmb.objVal))

assign[1] 1.0
assign[3] 1.0
assign[5] 1.0
region_in4[1] 1.0
region_in4[2] 1.0
region_in4[3] 1.0
region_in4[4] 1.0
region_in4[5] 1.0
region_in4[6] 1.0
Total ambulance: 3.0


still choosing location 1,3,5 and still only 1 ambulance for each